# Initializing the spark application

In [1]:
%%configure
{"conf": {
    "spark.app.name": "miaou_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8919,application_1589299642358_3451,pyspark,idle,Link,Link,
8973,application_1589299642358_3509,pyspark,dead,Link,Link,
8983,application_1589299642358_3520,pyspark,idle,Link,Link,
8996,application_1589299642358_3534,pyspark,idle,Link,Link,
9003,application_1589299642358_3541,pyspark,idle,Link,Link,
9028,application_1589299642358_3573,pyspark,idle,Link,Link,
9035,application_1589299642358_3582,pyspark,idle,Link,Link,
9045,application_1589299642358_3590,pyspark,idle,Link,Link,
9054,application_1589299642358_3599,pyspark,idle,Link,Link,
9055,application_1589299642358_3601,pyspark,idle,Link,Link,


In [2]:
# Initialization

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9085,application_1589299642358_3645,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.sql.functions import pandas_udf, col, collect_list
import numpy as np
import geopy.distance
import pyspark.sql.functions as f
from pyspark.sql.functions import explode, col, lit
from pyspark.sql import Window
from datetime import datetime
from math import ceil
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, array
from pyspark.sql.functions import to_timestamp, date_format
from math import ceil
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType, ArrayType, FloatType
from scipy import stats
import numpy as np
from pyspark.sql.functions import pandas_udf, col, collect_list
from pyspark.sql.types import IntegerType
from scipy import stats
import numpy as np
from math import ceil
from datetime import datetime, timedelta
from pyspark.sql.types import StringType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Setting up the data

Import all timetable and calculate distance of stops from Zurich station

In [6]:
stops = spark.read.csv('/data/sbb/timetables/csv/stops/2019/05/14/stops.txt', header=True)
stop_times = spark.read.csv('/data/sbb/timetables/csv/stop_times/2019/05/14/stop_times.txt', header=True)
trips=spark.read.csv('/data/sbb/timetables/csv/trips/2019/05/14/trips.txt', header=True)
calendar=spark.read.csv('/data/sbb/timetables/csv/calendar/2019/05/14/calendar.txt', header=True)
routes=spark.read.csv('/data/sbb/timetables/csv/routes/2019/05/14/routes.txt', header=True)

@f.udf("float")
def vincenty_udf(x, y):
    return geopy.distance.vincenty(x, y).km

coords_zurich = f.struct(f.lit(47.378177), f.lit(8.540192))
stops=stops.withColumn('Dist_to_zurich',vincenty_udf(coords_zurich, f.struct(stops.stop_lat.cast("double"),stops.stop_lon.cast("double"))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create Walking table that links stations which are less than 500m away from each other

In [4]:
stops_table = stops.filter(stops.Dist_to_zurich<15)
stops_table = stops_table.withColumn('stop_id_s',f.substring(stops_table.stop_id,0,7))
stops_table = stops_table.dropDuplicates(["stop_id_s"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_stop = stops_table.toPandas()
df_stop["Dist_to_station"]=np.zeros(len(df_stop))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def close_stations(lat, long, df):
    
    coords = (lat, long)
    
    df["Dist_to_station"] = df.apply(lambda x : geopy.distance.vincenty(coords, (x.stop_lat, x.stop_lon)).km, axis=1)
    return df[df["Dist_to_station"]<0.5]["stop_id_s"].tolist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_stop["close_stations"] = df_stop.apply(lambda x : close_stations(x.stop_lat, x.stop_lon, df_stop), axis=1)
df_stop = df_stop[["stop_id_s", "stop_lat", "stop_lon", "close_stations"]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
walking_table = spark.createDataFrame(df_stop).withColumn("close_stations", explode("close_stations"))
stops_table = stops_table.select(col("stop_id_s").alias("close_stop_id_s"), col("stop_lat").alias("close_stop_lat"), col("stop_lon").alias("close_stop_lon"))
walking_table = walking_table.join(stops_table, walking_table.close_stations==stops_table.close_stop_id_s)
walking_table = walking_table.withColumn("time", vincenty_udf(f.struct(walking_table.stop_lat.cast("double"), walking_table.stop_lon.cast("double")), f.struct(walking_table.close_stop_lat.cast("double"), walking_table.close_stop_lon.cast("double"))))
walking_table = walking_table.filter(walking_table.time>0)
walking_table = walking_table.withColumn("time", (20*walking_table.time).cast("int")).select("stop_id_s", "close_stop_id_s", "time")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
walking_table.write.parquet("/user/eckes/walking_table.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Join all timetables and add next stop to calculate travel time between two stops

In [7]:
#join all timetables
df=stop_times.join(stops,"stop_id")\
.join(trips,"trip_id")\
.join(calendar,"service_id")\
.join(routes,"route_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#filter stations less than 15km from Zurich station and for trains that operates every weekday. Also ignore trains outside rush hours
df=df.filter((df.Dist_to_zurich<=15)&(df.monday=='1')&(df.tuesday=='1')&(df.wednesday=='1')&(df.thursday=='1')&(df.friday=='1'))\
.filter((f.hour('departure_time')>6)&(f.hour('arrival_time')<19))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#New stop_id=parent station if available else it's the stop id
df=df.withColumn('stop_id_s',f.substring(df.stop_id,0,7)).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#get next station info
df=df.withColumn("next_stop_id_s", f.lead(df.stop_id_s).over(Window.partitionBy(df.trip_id).orderBy(df.arrival_time,df.direction_id,df.stop_sequence.cast("int"))))\
.withColumn("next_arrival_time", f.lead(df.arrival_time).over(Window.partitionBy(df.trip_id).orderBy(df.arrival_time,df.direction_id,df.stop_sequence.cast("int"))))\
.withColumn("next_departure_time", f.lead(df.departure_time).over(Window.partitionBy(df.trip_id).orderBy(df.arrival_time,df.direction_id,df.stop_sequence.cast("int"))))\
.withColumn("next_stop_lat", f.lead(df.stop_lat).over(Window.partitionBy(df.trip_id).orderBy(df.arrival_time,df.direction_id,df.stop_sequence.cast("int"))))\
.withColumn("next_stop_lon", f.lead(df.stop_lon).over(Window.partitionBy(df.trip_id).orderBy(df.arrival_time,df.direction_id,df.stop_sequence.cast("int"))))\
.withColumn("next_stop_name", f.lead(df.stop_name).over(Window.partitionBy(df.trip_id).orderBy(df.arrival_time,df.direction_id,df.stop_sequence.cast("int"))))\
.withColumn("next_trip_id", f.lead(df.trip_id).over(Window.partitionBy(df.trip_id).orderBy(df.arrival_time,df.direction_id,df.stop_sequence.cast("int"))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#calculate travel time
df=df.withColumn('Travel_time',(df.next_arrival_time.cast('timestamp').cast('long')-df.arrival_time.cast('timestamp').cast('long'))/60).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Reading the SBB data in orc format
sbb = spark.read.orc('/data/sbb/orc/istdaten') 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Cleaning the SBB data

- The SBB data has a few columns that we do not need, such as `betreiber_id, betreiber_abk, betreiber_name, linien_text, verkehrsmittel_text`, so we remove them.

- We assume that the trips that are not part of the regular schedule (`zusatzfahrt_tf = True`) or those that were failed, cancelled, or not completed (`faellt_aus_tf = False`), as well as those that do not stop (`durchfahrt_tf = False`) should not be taken into consideration as they are corner cases and cannot be used to predict usual delays.

- The `an/ab_prognose_status` indicates whether the data was real (`REAL`), forecast (`PROGNOSE`), estimated (`GESCHAETZT`),  or unknown (`UNBEKANNT`).  For the `an_prognose_status` for example, the following table shows the distribution of the sbb data:

	|`GESCHAETZT`| 90076185 |
    
	|`PROGNOSE`| 603610258 |
    
	|  `UNBEKANNT` | 12872226 |
    
	|  `REAL`| 128140212|

	The `REAL` data is obviously to be trusted, however, it constitues only 8% of the data. Consequently, we decide that the `GESCHAETZT` should also be taken into account in our predictions. 
    
- However, we noticed that Trams only have a `PROGNOSE` status. Thus, in order to keep the tram-related data, we decide to keep the rows where the transport type is 'Tram' and the `an/ab_prognose_status` is `PROGNOSE`.

In [4]:
columns_to_drop = ['betreiber_id', 'betreiber_abk', 'betreiber_name', 'linien_text', 'verkehrsmittel_text']
sbb = sbb.drop(*columns_to_drop)

# We remove all trips where ZUSATZFAHRT_TF is true (not part of the regular schedule)
sbb = sbb.filter(sbb.zusatzfahrt_tf == 'false')
sbb = sbb.drop('zusatzfahrt_tf')

# We remove all trips where FAELLT_AUS_TF is true (trip failed, cancelled or not completed)
sbb = sbb.filter(sbb.faellt_aus_tf == 'false')
sbb = sbb.drop('faellt_aus_tf')

# We only keep trips when AN_PROGNOSE_STATUS and AB_PROGNOSE_STATUS are REAL or GESCHAETZT (forecast) or PROGNOSE (estimated)
sbb = sbb.filter((sbb.an_prognose_status == 'REAL') | (sbb.an_prognose_status == 'GESCHAETZT') |\
                 ((sbb.produkt_id == 'Tram') & (sbb.an_prognose_status == 'PROGNOSE')))
# sbb = sbb.filter((sbb.an_prognose_status == 'REAL') | (sbb.an_prognose_status == 'GESCHAETZT'))
sbb = sbb.drop('an_prognose_status')
sbb = sbb.filter((sbb.ab_prognose_status == 'REAL') | (sbb.ab_prognose_status == 'GESCHAETZT') |\
                 ((sbb.produkt_id == 'Tram') & (sbb.ab_prognose_status == 'PROGNOSE')))
# sbb = sbb.filter((sbb.ab_prognose_status == 'REAL') | (sbb.ab_prognose_status == 'GESCHAETZT'))
sbb = sbb.drop('ab_prognose_status')

# When the means of transport drives straight through, drop the rows (durchfahrt_tf = true)
sbb = sbb.filter(sbb.durchfahrt_tf == 'false')
sbb = sbb.drop('durchfahrt_tf')

# When the stop is the start or end of a journey, the corresponding columns will be empty (ANKUNFTSZEIT/ABFAHRTSZEIT).
# We remove rows where the actual time data is empty
sbb = sbb.filter((sbb.an_prognose != '') & (sbb.ab_prognose != '')) 
# We remove rows where ankunftszeit is empty (the stop is the start of a journey so we assume there is no delay for the departure)
sbb = sbb.filter((sbb.ankunftszeit != ''))  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Translating the column names to english
sbb = sbb.select(col("betriebstag").alias("date"), col("fahrt_bezeichner").alias("trip_id"),\
                 col("linien_id").alias("line_id"), col("produkt_id").alias("transport_type"), col("umlauf_id").alias("circuit_id")\
                , col("haltestellen_name").alias("stop_name"), col("ankunftszeit").alias("arrival_time"), col("an_prognose").alias("actual_arrival_time")\
                , col("abfahrtszeit").alias("departure_time"), col("ab_prognose").alias("actual_departure_time"),col("bpuic")) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Only keep stations in a radius of 15km of Zürich
# We simply join sbb with df since df has been filtered and only contains stations in a radius of 15km of Zürich

# In particular, we remove any platform numbers from the stop_id if df 
gate_function = udf(lambda x: x.split(':')[0] if x != None else '', StringType())
df = df.withColumn('stop_id_clean', gate_function('stop_id'))
df = df.withColumn('next_stop_id_clean', gate_function('next_stop_id_s')) 

sbb = sbb.join(df, sbb.bpuic == df.stop_id_clean, "left_semi") 

In [ ]:
# Getting the next stop name
sbb=sbb.withColumn("next_stop_name", f.lead(sbb.stop_name).over(Window.partitionBy(sbb.trip_id).orderBy(sbb.arrival_time)))\
.withColumn("next_stop_bpuic", f.lead(sbb.bpuic).over(Window.partitionBy(sbb.trip_id).orderBy(sbb.arrival_time)))

In [ ]:
# The arrival & departure times columns contain dates so we remove those
# We already have a "date" column sbb
sbb = sbb.withColumn('arrival_time',f.substring(sbb.arrival_time,12,15)).\
withColumn('actual_arrival_time',f.substring(sbb.actual_arrival_time,12,15)).\
withColumn('departure_time',f.substring(sbb.departure_time,12,15)).\
withColumn('actual_departure_time',f.substring(sbb.actual_departure_time,12,15)) 

In [ ]:
# Add ":00" (seconds) to the time columns because they are in HH:MM format and we want them in the HH:MM:SS fmt
seconds_function = udf(lambda x: x + ":00", StringType())
sbb = sbb.withColumn('arrival_time', seconds_function('arrival_time'))
sbb = sbb.withColumn('departure_time', seconds_function('departure_time')) 

In [ ]:
# Remove weekends
sbb = sbb.withColumn("input_timestamp",to_timestamp(col("date"),"dd.MM.yyyy")).withColumn("week_day_number", date_format(col("input_timestamp"), "u")) #add day column
sbb = sbb.filter(sbb.week_day_number < 6)
sbb = sbb.drop(*['input_timestamp', 'week_day_number'])

In [ ]:
# Remove hours not between 6AM and 7PM
sbb = sbb.filter((f.hour('departure_time')>6)&(f.hour('arrival_time')<19))

# Computing the delays

We now have the scheduled times (`arrival_time` column) and the actual times (`actual_arrival_time` column). We would like to create a new column `arrival_delay` that shows the arrival delay for each trip.

In [4]:
# Define function to compute the delay: computes the difference between actual time and scheduled time
def compute_delay(actual, planned):
    #planned += ':00'
    format_ = '%H:%M:%S'
    actual_time = datetime.strptime(actual, format_)
    planned_time = datetime.strptime(planned, format_)
    
    if (actual_time < planned_time):
        return 0
    
    delay = actual_time - planned_time
    delay = int(ceil(delay.seconds/60.0))
    return delay

delay_function = udf(lambda x: compute_delay(x[0], x[1]), IntegerType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Add arrival & departure delay columns to sbb
sbb = sbb.withColumn('arrival_delay', delay_function(array('actual_arrival_time', 'arrival_time')))
sbb = sbb.withColumn('departure_delay',delay_function(array('actual_departure_time', 'departure_time')))  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sbb_filtered = sbb.withColumn('arrival_hour', f.hour('arrival_time'))

In [ ]:
#sbb_filtered.write.format("orc").save("/user/ellouz/sbb_filtered_final.orc")

In [3]:
sbb_filtered = spark.read.orc("/user/ellouz/sbb_filtered_final.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
sbb_filtered.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+-------+--------------+----------+----------------+------------+-------------------+--------------+---------------------+-------+----------------+---------------+-------------+---------------+------------+
|      date|        trip_id|line_id|transport_type|circuit_id|       stop_name|arrival_time|actual_arrival_time|departure_time|actual_departure_time|  bpuic|  next_stop_name|next_stop_bpuic|arrival_delay|departure_delay|arrival_hour|
+----------+---------------+-------+--------------+----------+----------------+------------+-------------------+--------------+---------------------+-------+----------------+---------------+-------------+---------------+------------+
|14.05.2019|85:11:14047:002|  14047|           Zug|          |      Effretikon|    12:22:00|           12:23:07|      12:23:00|             12:23:36|8503305|     Bassersdorf|        8503307|            2|              1|          12|
|14.05.2019|85:11:14047:002|  14047|           Zug|          |  

In [12]:
# We add the arrival_hour_df column in case we'd like to use the second approach
df = df.withColumn('arrival_hour_df', f.hour('arrival_time'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df.groupBy(df.route_desc).count().show() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+
|   route_desc| count|
+-------------+------+
|          TGV|    11|
|     Eurocity|    50|
|Standseilbahn|  1815|
| RegioExpress|   166|
|       S-Bahn| 18073|
| Luftseilbahn|   176|
|         Taxi|   133|
|         Tram| 63415|
|          ICE|    18|
|          Bus|195044|
|       Schiff|  1079|
|   InterRegio|   794|
|    Intercity|   518|
+-------------+------+

In [ ]:
# df cleaning: replace transport type with category name 
df = df.replace(['TGV', 'Eurocity', 'RegioExpress', 'S-Bahn', 'ICE', 'InterRegio', 'Intercity'], 'Zug', 'route_desc')\
.replace(['Standseilbahn', 'Luftseilbahn', 'Taxi', 'Schiff'], '', 'route_desc')

In [ ]:
#df.write.format("orc").save("/user/ellouz/df_for_merge.orc")

In [4]:
df = spark.read.orc("/user/ellouz/df_for_merge.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Fitting a distribution to the delays: two approaches
[Several studies](https://ascelibrary.org/doi/pdf/10.1061/%28ASCE%29TE.1943-5436.0000724) recommend the **lognormal** distribution as the descriptor of day-to-day public transport travel time. Since our data involves trains, buses, and other transport types, we find this distribution fitting to our study and decide to model the travel times using it.

To that end, we constitute groups of delays. 
1) A **first approach** would be to create a group of delays for each transport type present in the SBB data. After that, we can fit the distribution on each group and get a set of parameters for each transport type. As we see below, there are 4 categories: Trams, Trains (Zug), Buses, and other types of transport.

In [6]:
sbb_filtered.groupBy('transport_type').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+
|transport_type|   count|
+--------------+--------+
|          Tram|30950230|
|           Zug| 5122675|
|           Bus|  320042|
|              |    2036|
+--------------+--------+

2) A **second approach** is to isolate the delays of each ( `stop_id, arrival_time, transport_type`) group. This means that we model each station at each arrival time and for each transport type. The benefit of this approach is that the delays become more specific to the station and to the time, especially if we transform `arrival_time` into `arrival_hour` to make bigger groups. Just like approach 1, we would get a set of parameters for each group.

Each approach has its pros and cons and in the following, both implementations are present. However, due to time constraints and because of the few bugs the second approach presented, we decided to adopt approach 1.

First we define a `compute_params` function that is common to both approaches. It takes a list of delays and use `stats.lognorm.fit` to retrieve the corresponding parameters.

In [10]:
#compute the lognorm parameter, return None if list is empty
#returns a list of 3 floats
def compute_params(l): 
    if len(l) > 0: 
        params = stats.lognorm.fit(l, loc = 0)
        return [float(x) for x in params] # need to convert to float because udf doesn't like numpy.float
    else:
        return None

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## First approach: unique parameters for each transport type
We can discard outliers by filtering out arrival delay > 100 minutes. Our reasoning was that when the delay was high, the user may have had the possibility to take another connection whose arrival time was smaller than the arrival time + arrival delay of the connection she was waiting on. Plus a high delay is unusual and usually out of control of the transport company (e.g, road closed because of an accident, storm destroying part of the railways,...).

##### Train delays

In [12]:
# Filter out when delay > 100 (outliers)
sbb_train = sbb_filtered.select(sbb_filtered.transport_type,sbb_filtered.arrival_delay).filter(sbb_filtered.transport_type == 'Zug').cache()
sbb_train = sbb_train.filter(sbb_train.arrival_delay < 100).cache()
group_arr_train = sbb_train.agg(collect_list('arrival_delay').alias('arr_delay_list'))
compute_params_function = udf(lambda x: compute_params(x),ArrayType(FloatType())) #x is the list of the delay of a group
group_arr_train = group_arr_train.withColumn('arr_params', compute_params_function('arr_delay_list'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
group_arr_train.select(group_arr_train.arr_params).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------+
|arr_params                        |
+----------------------------------+
|[0.47517332, -0.9588776, 2.308198]|
+----------------------------------+

##### Tram delays

In [17]:
#sbb_trams_sample = sbb_trams.sample(False, 0.01, seed=10)
# sbb_trams_sample.write.format("orc").save("/user/ellouz/trams_sample.orc")
sbb_trams_sample = spark.read.orc("/user/ellouz/trams_sample.orc")
sbb_trams_sample = sbb_filtered.select(sbb_filtered.transport_type,sbb_filtered.arrival_delay).filter(sbb_filtered.transport_type == 'Tram')
sbb_trams_sample = sbb_trams_sample.filter(sbb_trams_sample.arrival_delay < 100)
group_arr_tram = sbb_trams_sample.agg(collect_list('arrival_delay').alias('arr_delay_list'))
compute_params_function = udf(lambda x: compute_params(x),ArrayType(FloatType())) #x is the list of the delay of a group
group_arr_tram = group_arr_tram.withColumn('arr_params', compute_params_function('arr_delay_list'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
group_arr_tram.select('arr_params').show(truncate=False)

##### Bus delays

In [11]:
sbb_buses = sbb_filtered.select(sbb_filtered.transport_type,sbb_filtered.arrival_delay).filter(sbb_filtered.transport_type == 'Bus').cache()
sbb_buses = sbb_buses.filter(sbb_buses.arrival_delay < 100).cache()
group_arr_bus = sbb_buses.agg(collect_list('arrival_delay').alias('arr_delay_list'))
compute_params_function = udf(lambda x: compute_params(x),ArrayType(FloatType())) #x is the list of the delay of a group
group_arr_bus = group_arr_bus.withColumn('arr_params', compute_params_function('arr_delay_list'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
group_arr_bus.select('arr_params').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+
|arr_params                          |
+------------------------------------+
|[0.56315994, -4.974291E-4, 1.668936]|
+------------------------------------+

##### Other transport types delays

In [13]:
sbb_others = sbb_filtered.select(sbb_filtered.transport_type,sbb_filtered.arrival_delay).filter(sbb_filtered.transport_type == '').cache()
sbb_others = sbb_others.filter(sbb_others.arrival_delay < 100).cache()
group_arr_others = sbb_others.agg(collect_list('arrival_delay').alias('arr_delay_list'))
compute_params_function = udf(lambda x: compute_params(x),ArrayType(FloatType())) #x is the list of the delay of a group
group_arr_others = group_arr_others.withColumn('arr_params', compute_params_function('arr_delay_list'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
group_arr_others.select('arr_params').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+
|arr_params                          |
+------------------------------------+
|[0.48570195, -0.91664267, 2.5085754]|
+------------------------------------+

In [ ]:
transport_dict = {'Zug': [0.47517332, -0.9588776, 2.308198], 'Bus': [0.56315994, -4.974291E-4, 1.668936], 
                  'Tram': [4.607556, 1.0, 5.1064895E-9], '':[0.48570195, -0.91664267, 2.5085754]}  

## Second approach: unique parameters for each (station_id, arrival_hour, transport_type) group

##### Computing the parameters of each group

In [ ]:
# Make a group for each ('stop_id', 'arrival_hour', 'transport_type')   tuple
grouped_delays_hour = sbb_filtered.groupBy('bpuic', 'arrival_hour', 'transport_type')  

In [ ]:
#https://stackoverflow.com/questions/40006395/applying-udfs-on-groupeddata-in-pyspark-with-functioning-python-example#40030740
group_arr_hour = grouped_delays_hour.agg(collect_list('arrival_delay').alias('arr_delay_list'))

compute_params_function = udf(lambda x: compute_params(x),ArrayType(FloatType())) #x is the list of the delay of a group
group_arr_hour = group_arr_hour.withColumn('arr_params', compute_params_function('arr_delay_list'))

In [ ]:
# group_arr_hour.write.format("orc").save("/user/ellouz/group_arr_hour.orc")

In [21]:
group_arr_hour = spark.read.orc("/user/ellouz/group_arr_hour.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
group_arr_hour.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+--------------+--------------------+--------------------+
|  bpuic|arrival_hour|transport_type|      arr_delay_list|          arr_params|
+-------+------------+--------------+--------------------+--------------------+
|8503010|          11|           Zug|[0, 0, 0, 0, 1, 0...|[7.2196875, 1.0, ...|
|8503016|          14|           Zug|[4, 1, 4, 1, 1, 1...|[9.560064, 1.0, 2...|
|8503053|           8|           Zug|[3, 5, 1, 3, 3, 2...|[12.998721, 1.0, ...|
|8503307|          15|           Zug|[2, 0, 1, 1, 2, 1...|[9.904795, 1.0, 1...|
|8503312|          13|           Zug|[1, 0, 1, 0, 1, 0...|[7.035858, 1.0, 0...|
+-------+------------+--------------+--------------------+--------------------+
only showing top 5 rows

##### Merging arrival delays parameters with df (general trips data)

Now that we have cleaned the sbb and df data, we merge sbb with df. We keep all entries of df since they contains all general trips. Sometimes a trip from df is not found in sbb: in this case, our strategy is to use the general expected delay for the corresponding means of transport computed in approach 1. For example, if there is a certain bus trip which does not have any corresponding trips in sbb, we use the parameters of the lognormal distribution for all bus delays in sbb and predict a delay for that bus trip.

In [ ]:
cond_hour = [group_arr_hour.bpuic == df.stop_id_clean, group_arr_hour.arrival_hour == df.arrival_hour_df, group_arr_hour.transport_type == df.route_desc]
left_joined_hour = df.join(group_arr_hour, cond_hour, "left") 

In [ ]:
# left_joined_hour.write.format("orc").save("/user/ellouz/left_joined_hour.orc")

In [14]:
left_joined_hour = spark.read.orc("/user/ellouz/left_joined_hour.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
left_joined_hour.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+--------------------+-------+------------+--------------+-------------+-----------+-------------+-------------------+----------------+----------------+-------------+--------------+--------------+---------------+---------------+------------+------+-------+---------+--------+------+--------+------+----------+--------+---------+----------------+---------------+----------+----------+---------+--------------+-----------------+-------------------+----------------+----------------+-------------------+--------------------+-----------+-------------+------------------+---------------+-------+------------+--------------+--------------------+--------------------+
|     route_id|service_id|             trip_id|stop_id|arrival_time|departure_time|stop_sequence|pickup_type|drop_off_type|          stop_name|        stop_lat|        stop_lon|location_type|parent_station|Dist_to_zurich|  trip_headsign|trip_short_name|direction_id|monday|tuesday|wednesday|thursday|friday|saturda

#### Predicting delay based on the probability given by the user

Now that we have computed the **parameters of the lognorm distribution** for the arrival delay of each entry of our dataframe, we decided to **predict a delay based on a probability**. 
For example if a user wants 80% certainty of her route, we assumed that it will need each leg of the final itinerary to have a certainty probability of 80% (similar to the weakest link in computer network). So to compute a delay with 80% certainty, we compute the **percent point function of the lognorm** that **given a probability, returns the maximum delay such that 80% of the possible delay are less or equal to this maximum delay**. With this reasoning it means that if we want 100% certainty, it will give back a delay equals to infinity. So the bigger the probability given by the user, the bigger the delay computed. 
To simplify our model, we precomputed the delay for 4 different probabilities from which the user can choose : 
- 80%
- 85%
- 90%
- 95%

We can create a `predict_delay` function that computes the delay given a percentile and a list of parameters for the distribution. 
For the second approach, the `predict_delay_2nd_approach` version takes into account the cases where trips are not found in the sbb data and uses the parameters of the transport type in those cases.

In [ ]:
def predict_delay(transport_type, q): 
    
    type_parameters = transport_dict[transport_type]
    shape, loc, scale = type_parameters[0], type_parameters[1], type_parameters[2]
        
    return int(ceil(stats.lognorm.ppf(q, shape, loc, scale)))


def predict_delay_2nd_approach(transport_type, parameters, q): 
#     If the parameters are already computed
    if parameters != None:
        shape, loc, scale = parameters[0], parameters[1], parameters[2]
#     Else use the Bus/Zug/Tram parameters
    else:
        type_parameters = transport_dict[transport_type]
        shape, loc, scale = type_parameters[0], type_parameters[1], type_parameters[2]
        
    return int(ceil(stats.lognorm.ppf(q, shape, loc, scale)))

In [ ]:
# First approach: by transport type
predict_delay_function = udf(lambda x, y: predict_delay(x, y),IntegerType())
joined_delay = left_joined_hour.withColumn('pred_delay_80', predict_delay_function('route_desc', lit(0.80)))\
.withColumn('pred_delay_85', predict_delay_function('route_desc', lit(0.85)))\
.withColumn('pred_delay_90', predict_delay_function('route_desc', lit(0.90)))\
.withColumn('pred_delay_95', predict_delay_function('route_desc', lit(0.95)))

# Second approach: by (station, hour, transport_type)
# predict_delay_function = udf(lambda x, y, z: predict_delay(x, y, z),IntegerType())
# joined_delay = left_joined_hour.withColumn('pred_delay_80', predict_delay_function('route_desc', 'arr_params', lit(0.80)))\
# .withColumn('pred_delay_85', predict_delay_function('route_desc', 'arr_params', lit(0.85)))\
# .withColumn('pred_delay_90', predict_delay_function('route_desc', 'arr_params', lit(0.90)))\
# .withColumn('pred_delay_95', predict_delay_function('route_desc', 'arr_params', lit(0.95)))

In [ ]:
# We remove unneeded columns
columns_to_drop = ['drop_off_type', 'stop_lat', 'stop_lon', 'Dist_to_zurich', 'trip_headsign', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday', 'sunday']
joined_delay = joined_delay.drop(*columns_to_drop)

While observing our results from the second approach, we found out that 8 entries of the pred_delay_95 columns had null values because the delay computed was too big. Therefore we replaced the null values with a delay of 100 minutes instead. We also found out that some delays computed were negative or way too big therefore we also considered them equal to 100 minutes in the function ```compute_new_arrival```. Our reasoning is the same than when we computed the general lognorm distribution for each transport type. We think it is reasonable to assume that a delay > 100 minutes is not SBB's fault and unusual, and really bad luck.

In [18]:
# Second approach related. This is not used in the first approach
# joined_delay = joined_delay.fillna({'pred_delay_95' : str(100)}) #we found out that 8 entries of this column were null

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#joined_delay.write.format("orc").save("/user/ellouz/joined_delay_final.orc")

In [17]:
joined_delay = spark.read.orc("/user/ellouz/joined_delay_final.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now that we have different delays that corresponds to different certainty degrees (or success probabilities), we can **add this delay to the arrival time** of a trip and then calculate the shortest path using this new arrival time as input to our graph. So the **time-dependent part of the shortest path was modeled in the parameters we gave to the algorithm**. Instead of answering to the question "I will have a delay of x with this train, so will I make it to that train that departs at hh:mm?", we simplified it by saying "My train will have a delay of x minutes, so I will arrive at hh:mm + x minutes instead of hh:mm which I now consider as the new arrival time of my train".

*Example*:

Let's say there is a train A that was supposed to arrive at 10:30. However, based on the previous delays we were able to compute, we fitted a lognorm distribution on it to get the lognorm parameters. Then we computed the maximal delay of the distribution such that 80% of the delay distribution is smaller or equal than this value, it gave us 3 minutes of delay. Now we did the same thing for 90% which gave us 4 minutes of delay.
Now we changed the arrival time to 10:33 and to 10:34 respectively. When the user selects the certainty she wants (80 or 90% in this example), we will change the arrival time of train A to the one corresponding to the certainty level. We do that for our entire dataset before constructing the graph that we will then feed to the shortest path algorithm.

In [ ]:
#add the delay to the arrival time
# delay is a str(delay_in_minutes)
def compute_new_arrival(delay, arrival_time):
    if((int(delay) < 0) | (int(delay) > 100)):
        delay = 100
    delay_dt = timedelta(minutes=int(delay))
    format_ = '%H:%M:%S'
    actual_time = datetime.strptime(arrival_time, format_)
    new_time = actual_time + delay_dt
    return str(new_time.time())

In [ ]:
compute_new_arrival_function = udf(lambda x: compute_new_arrival(x[0], x[1]), StringType())
joined_delay = joined_delay.withColumn('arrival_time_80', compute_new_arrival_function(array('pred_delay_80','arrival_time')))\
.withColumn('arrival_time_85', compute_new_arrival_function(array('pred_delay_85','arrival_time')))\
.withColumn('arrival_time_90', compute_new_arrival_function(array('pred_delay_90','arrival_time')))\
.withColumn('arrival_time_95', compute_new_arrival_function(array('pred_delay_95','arrival_time')))\

In [ ]:
# In case of first approach (delays by transport type)
#joined_delay.write.format("orc").save("/user/ellouz/delayed_arrival_times_basic.orc")

In [ ]:
# In case of second approach (by station, hour, trnasport_type)
#joined_delay.write.format("orc").save("/user/ellouz/df_final9.orc")

In [4]:
# 1st approach
df_basic = spark.read.orc("/user/ellouz/delayed_arrival_times_basic.orc")
important_columns = ['stop_id_s', 'trip_id', 'arrival_time', 'departure_time', 'next_stop_id_s', 'next_trip_id',\
                'next_arrival_time', 'next_departure_time', 'Travel_time', 'stop_name', 'arrival_time_80',\
               'arrival_time_85', 'arrival_time_90', 'arrival_time_95', 'arrival_time_99']
edges=df_basic.select(important_columns).fillna({'Travel_time':0})
#edges.write.parquet("/user/eckes/edges.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# 2nd approach
# df_final = spark.read.orc("/user/ellouz/df_final9.orc")
# important_columns = ['stop_id_s', 'trip_id', 'arrival_time', 'departure_time', 'next_stop_id_s', 'next_trip_id',\
#                 'next_arrival_time', 'next_departure_time', 'Travel_time', 'stop_name', 'arrival_time_80',\
#                'arrival_time_85', 'arrival_time_90', 'arrival_time_95']
# edges=df_final.select(important_columns).fillna({'Travel_time':0})
# edges.write.parquet("/user/abourjei/edges_2.parquet")